In [2]:
import csv
from qiskit.quantum_info import Operator
from qiskit import QuantumCircuit, transpile

qasm_filename = "qf21_n15_transpiled.qasm"

# Add this function to round near-zero values to exactly zero
def round_near_zero(matrix, threshold=1e-15):
    return [
        [
            complex(
                0 if abs(element.real) < threshold else element.real,
                0 if abs(element.imag) < threshold else element.imag
            )
            for element in row
        ]
        for row in matrix
    ]

# Load the circuit from the QASM file
circuit = QuantumCircuit.from_qasm_file(qasm_filename)
circuit.remove_final_measurements()

# Map qubits to their indices
qubit_indices = {qubit: idx for idx, qubit in enumerate(circuit.qubits)}

# Prepare data for CSV
output_data = []

# Extract the operational matrices of each gate along with qubit information
for i, instruction in enumerate(circuit.data):
    gate = instruction.operation  # The gate (operation)
    if gate.name != "measure" and gate.name !="barrier":
        qargs = instruction.qubits  # The qubits the gate acts on
        matrix = round_near_zero(Operator(gate).data) # Get the matrix representation
        
        # Collect gate information
        gate_info = [f"Gate {i + 1}", gate.name]
        if gate.num_qubits == 1:
            gate_info.append(f"") #empty control qubit
            gate_info.append(f"{qubit_indices[qargs[0]]}") #target only
        elif gate.num_qubits == 2:
            gate_info.append(f"{qubit_indices[qargs[0]]}") #control qubit
            gate_info.append(f"{qubit_indices[qargs[1]]}") #target qubit
        
        # Add matrix data row-wise
        gate_info.append(matrix)  # Convert matrix to list for CSV compatibility
        print(matrix)
        output_data.append(gate_info)

# Write the data to a CSV file
with open('quantum_circuit_gates.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write headers
    writer.writerow(["Gate Number", "Gate Name", "Control Qubit", "Target Qubit","Matrix", circuit.num_qubits])
    # Write data rows
    writer.writerows(output_data)

print("Data written to quantum_circuit_gates.csv")

[[(0.7071067811865476-0.7071067811865476j), 0j], [0j, (0.7071067811865476+0.7071067811865476j)]]
[[(0.5+0.5j), (0.5-0.5j)], [(0.5-0.5j), (0.5+0.5j)]]
[[(0.7071067811865476-0.7071067811865476j), 0j], [0j, (0.7071067811865476+0.7071067811865476j)]]
[[(0.7071067811865476-0.7071067811865476j), 0j], [0j, (0.7071067811865476+0.7071067811865476j)]]
[[(0.5+0.5j), (0.5-0.5j)], [(0.5-0.5j), (0.5+0.5j)]]
[[(0.7071067811865476-0.7071067811865476j), 0j], [0j, (0.7071067811865476+0.7071067811865476j)]]
[[(0.7071067811865476-0.7071067811865476j), 0j], [0j, (0.7071067811865476+0.7071067811865476j)]]
[[(0.5+0.5j), (0.5-0.5j)], [(0.5-0.5j), (0.5+0.5j)]]
[[(0.7071067811865476-0.7071067811865476j), 0j], [0j, (0.7071067811865476+0.7071067811865476j)]]
[[(0.7071067811865476-0.7071067811865476j), 0j], [0j, (0.7071067811865476+0.7071067811865476j)]]
[[(0.5+0.5j), (0.5-0.5j)], [(0.5-0.5j), (0.5+0.5j)]]
[[(0.7071067811865476-0.7071067811865476j), 0j], [0j, (0.7071067811865476+0.7071067811865476j)]]
[[(0.7071067